# TP : Génération augmentée de récupération (RAG) et agents

Dans ce projet, le but va être de construire un système de RAG complet, à partir d'outils et de modèles libres existants.

## Installation et importation des bibliothèques

On installe les bibliothèques nécessaires : LangChain pour l'orchestration et FAISS pour la base de données vectorielle, ainsi que le transformers et gdown.

In [ ]:
!pip install -U langchain-community
!pip install faiss-gpu-cu12
!pip install datasets
!pip install langchain-text-splitters
!pip install sentence-transformers
!pip install faiss-cpu
!pip install gdown

## Importation des bibliothèques

Configuration de l'environnement et suppression des logs verbeux pour garder la sortie propre.

In [ ]:
import logging
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer
from datasets import load_dataset

from langchain_community.docstore.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from smolagents import InferenceClientModel

## Collection de documents (Wikipedia)

On charge le jeu de données Wikipedia, crée des objets Document avec les métadonnées de la source, puis on les découpe en morceaux pour l'indexation.

In [ ]:
# 1. Chargement du jeu de données (10 premiers articles pour l'exemple de création)
ds = load_dataset("Sketched33/Cities_Wikipedia_Information", split="train[:10]")

# 2. Création des documents avec métadonnées
source_docs = []
for doc in ds:
    source_docs.append(Document(
        page_content=doc["wikipedia_content"],
        metadata={"source": "Wikipedia - " + doc["city_name"]}
    ))

# 3. Configuration du découpeur (Splitter)
embedding_model_name = "thenlper/gte-small"
tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer,
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
)

# 4. Découpage des documents
docs_processed = []
docs_added = set()
for doc in tqdm(source_docs, desc="Découpage des documents"):
    new_docs = text_splitter.split_documents([doc])
    for new_doc in new_docs:
        if new_doc.page_content not in docs_added:
            docs_added.add(new_doc.page_content)
            docs_processed.append(new_doc)

print(f'\nTotal : {len(docs_processed)} sous-documents créés.')

## Indexation et Chargement de la Base Vectorielle

Pour gagner du temps et avoir une base de connaissances plus large, on télécharge ensuite un index pré-calculé de 3000 articles.

In [ ]:
# Chargement du modèle d'embeddings
embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)

# Création de l'index pour nos 10 documents (Démonstration)
vectordb_demo = FAISS.from_documents(docs_processed, embedding=embedding_model, distance_strategy='cosine')
print("Index de démonstration créé avec succès.")

# --- CHARGEMENT DE LA BASE DE DONNÉES COMPLÈTE (3000 articles) ---
# On utilise une base plus large pour le système final
!gdown --id 1a2WziQTnghzMuGfd5oHKynMSMjUC3kBJ -O index.zip
!mkdir -p faiss_index
!unzip -o index -d faiss_index

vectordb = FAISS.load_local(
    "faiss_index",
    embedding_model, 
    allow_dangerous_deserialization=True
)
print("Base de données chargée avec succès !")

## Configuration du LLM

On va utiliser le modèle Qwen/Qwen2.5-72B-Instruct via l'API d'inférence Hugging Face.

In [ ]:
# REMARQUE : Il faut rapmlacer "hf_XXX" par propre token Hugging Face
api_token = "hf_XXXXXXXXXXXXXXXXXX"

# Initialisation du modèle
llm_engine = InferenceClientModel(
    model_id="Qwen/Qwen2.5-72B-Instruct", 
    token=api_token
)

## Système Complet (RAG Loop)

On crée la boucle principale. L'utilisateur pose une question, le système cherche les 5 documents les plus pertinents dans la base FAISS, et le LLM génère une réponse basée sur ces sources.

In [ ]:
# Prompt template pour le RAG
rag_prompt = """
You are un AI assistant. Answer the user's questions using nothing but the following documents.
If the answer can't be found in the documents, say "I don't know".
For every answer you should provide a source file from which you got the information with the link to the coresponding wikipedia page.

CONTEXT :
Document 1 (Source: {src1}) : {doc1}
Document 2 (Source: {src2}) : {doc2}
Document 3 (Source: {src3}) : {doc3}
Document 4 (Source: {src4}) : {doc4}
Document 5 (Source: {src5}) : {doc5}

User's question: {query}

Answer :
"""

print("Assistant: Hi! How can I help you? (Type 'Exit' to quit)")
user_query = input('User: ')

messages = []

while user_query != "Exit":
    # 1. Recherche (Retrieval)
    sorted_docs = vectordb.similarity_search(user_query, k=5)
    
    # Préparation des variables pour le prompt
    context_vars = {
        "query": user_query,
        "doc1": sorted_docs[0].page_content, "src1": sorted_docs[0].metadata['source'],
        "doc2": sorted_docs[1].page_content, "src2": sorted_docs[1].metadata['source'],
        "doc3": sorted_docs[2].page_content, "src3": sorted_docs[2].metadata['source'],
        "doc4": sorted_docs[3].page_content, "src4": sorted_docs[3].metadata['source'],
        "doc5": sorted_docs[4].page_content, "src5": sorted_docs[4].metadata['source'],
    }
    
    # 2. Construction du prompt
    full_prompt = rag_prompt.format(**context_vars)
    
    # 3. Génération (Generation)
    # On garde un historique basique pour l'affichage, mais ici le RAG est "stateless" à chaque tour pour simplifier
    messages = [{"role": "user", "content": full_prompt}]
    answer = llm_engine(messages).content
    
    print(f'\nAssistant: {answer}\n')
    
    # Tour suivant
    user_query = input('User: ')